# 将爬取的文本数据处理为ChatGLM3对话格式
1. 获取文本：爬取的文本数据存储在MySQL中，连接MySQL获取文本数据，Text(id, title, content, type)
2. 构造数据集：将type和title拼接起来作为prompt，划分训练集(80%)、验证集(20%)、测试集(20%)
3. 格式处理：将数据集处理为ChatGLM3对话格式，保存为json文件

## 1. 获取文本
爬取的文本数据存储在MySQL中，连接MySQL获取文本数据，Text(id, title, content, type)

总记录数：4893

In [2]:
import pymysql

In [3]:
HOST = 'localhost'
USER = 'root'
PASSWORD = '020411'
DATABASE = 'maxdb'

In [4]:
connection = pymysql.connect(host=HOST, user=USER, password=PASSWORD, db=DATABASE)
cursor = connection.cursor()

In [5]:
with connection, cursor:
    cursor.execute('select title, content, type from text;')
    results = cursor.fetchall()
len(results)

4893

## 2. 构造数据集
将type和title拼接起来作为prompt，划分训练集(80%)、验证集(20%)、测试集(20%)


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
data = pd.DataFrame(results, columns=['title', 'content', 'type'])
data.head()

,title,content,type
0,关于评选燕山大学2020/2021学年度优秀博士硕士学位论文暨推荐河北省优秀博士硕士学位论文的通知,院属各系部、研究生导师及相关研究生：学位论文质量是衡量研究生培养质量的重要标准。为进一步鼓励...,通知公告
1,关于机械工程学院2021级本科生专业分流绩点加分的公示,根据《机械工程学院2021级大类招生专业分流方案》相关要求，通过学生申报，学院核实，现对20...,通知公告
2,关于机械工程学院2021级大类招生专业分流结果的公示,2021级机械类专业本科生、院属各单位：根据《机械工程学院2021级大类招生专业分流方案》的...,通知公告
3,关于做好2022级硕士研究生与导师互选工作的通知,院属各系部、硕士研究生导师及2022级硕士研究生：根据研究生院“关于做好2022级硕士研究生...,通知公告
4,关于机械工程学院2019级本科生推荐免试研究生加分学生名单（创新创业竞赛获奖部分）的通知,机械工程学院相关同学：根据《燕山大学大学生创新创业竞赛组织与管理办法》（燕大校字[2021]...,通知公告


In [8]:
for idx, row in data.iterrows():
    title = f"{row['type']}#{row['title']}"
    data.loc[idx, 'title'] = title
data.drop(columns=['type'], inplace=True)
data.head()

,title,content
0,通知公告#关于评选燕山大学2020/2021学年度优秀博士硕士学位论文暨推荐河北省优秀博士硕...,院属各系部、研究生导师及相关研究生：学位论文质量是衡量研究生培养质量的重要标准。为进一步鼓励...
1,通知公告#关于机械工程学院2021级本科生专业分流绩点加分的公示,根据《机械工程学院2021级大类招生专业分流方案》相关要求，通过学生申报，学院核实，现对20...
2,通知公告#关于机械工程学院2021级大类招生专业分流结果的公示,2021级机械类专业本科生、院属各单位：根据《机械工程学院2021级大类招生专业分流方案》的...
3,通知公告#关于做好2022级硕士研究生与导师互选工作的通知,院属各系部、硕士研究生导师及2022级硕士研究生：根据研究生院“关于做好2022级硕士研究生...
4,通知公告#关于机械工程学院2019级本科生推荐免试研究生加分学生名单（创新创业竞赛获奖部分）的通知,机械工程学院相关同学：根据《燕山大学大学生创新创业竞赛组织与管理办法》（燕大校字[2021]...


In [9]:
data = data.sample(frac=1).reset_index(drop=True)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=200)
train_data, valid_data = train_test_split(train_data, test_size=0.25, random_state=200)
len(train_data), len(valid_data), len(test_data)

(2935, 979, 979)

In [10]:
train_data.head()

,title,content
3649,新闻动态#我院学生在2014年全国大学生机械产品数字化设计大赛中获得优异成绩,2014年5月16日—18日，由国家级实验教学示范中心联席会机械学科组承办的“全国大学生机械...
3834,新闻动态#机械工程学院召开2021级本科生安全信息员会议,为进一步增强学生安全意识，提高学生信息处理能力，营造良好校园环境，机械工程学院2021级本科...
11,新闻动态#学院成功举办第四届校园舞台剧大赛,5月9日晚7：00，由机械工程学院团委举办的第四届校园舞台剧大赛——“携手文明 舞动奥运”—...
1681,新闻动态#我院成功举办暑期社会实践宣讲会,大学生暑期社会实践活动宣讲会成功召开学生小组代表展示他们的暑期社会实践成果2013年10月1...
4504,通知公告#机械工程学院教职工元旦联欢活动通知,为了感谢学院广大教职工一年以来的辛勤工作，学院近日将举行全院辞旧岁迎新年联欢活动，请全院教职...


## 3. 格式处理
将数据集处理为ChatGLM3对话格式，保存为json文件

使用仅微调对话能力的格式，ChatGLM3对话格式如下
```json
[
  {
    "conversations": [
      {
        "role": "system",
        "content": "<system prompt text>"
      },
      {
        "role": "user",
        "content": "<user prompt text>"
      },
      {
        "role": "assistant",
        "content": "<assistant response text>"
      }
    ]
  }
]
```

In [11]:
import json
from pathlib import Path

In [12]:
train_file = Path('train.json')
valid_file = Path('valid.json')
test_file = Path('test.json')

In [15]:
def transform_and_save(data, path):
    conversations = []
    for _, row in data.iterrows():
        title = row['title']
        content = row['content']
        prompt = {'role': 'user', 'content': title}
        answer = {'role': 'assistant', 'content': content}
        conversations.append(prompt)
        conversations.append(answer)
    conversations = [{'conversations': conversations}]
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(conversations, f, ensure_ascii=False, indent=4)

In [16]:
transform_and_save(train_data, train_file)
transform_and_save(valid_data, valid_file)
transform_and_save(test_data, test_file)